# 환경부 보도자료

## 패키지 로드

In [2]:
import pandas as pd
import re
import konlpy
from konlpy.tag import Mecab
from collections import Counter
import nltk
#----------------------------
mecab=Mecab()

## 데이터 불러오기

In [3]:
df=pd.read_csv('env_news_df_clean.csv',index_col=0)

In [4]:
df.head()

,date,department,title,text
0,2020-09-29,생물다양성과,공동참고철새도래지 10개소 89종 28277개체 확인,['농식품부는 최근 겨울철새 도래현황 조사 결과환경부 대표적인 겨울철새인 오리기러기...
1,2020-09-29,환경보건정책과,가습기살균제 피해 300명 추가 인정총 3284명 지원,['제19차 가습기살균제 피해구제위원회 개최 특별법 개정 후 첫 회의 개정법에 따른...
2,2020-09-29,기후경제과,제3차 계획기간20212025 국가 배출권 할당계획 마련,['계획기간 배출권거래제 적용 업체들의 배출허용총량을 연평균 6억 970만톤으로 설...
3,2020-09-29,녹색전환정책과,환경부 추석 대비 중소환경기업 지원에 힘쓴다,['중소영세기업 추석 전후 재정지원 박차 융자 신속 집행 재활용 선별업체 추가 지원...
4,2020-09-28,심의지원소통팀,참고유역물관리위원회 영산강 2개보 처리방안 의견 제시,['승촌보는 물이용대책을 추진한 후 상시개방으로 결정 죽산보는 해체를 제안하나 개방...


## 전처리 보완

In [5]:
def clean_str(text):
    text=re.sub('[a-z]','',text)
    text=re.sub('[A-Z]','',text)
    pattern='붙임.*끝.'
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[0-9]+'         # 숫자제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[0-9]+[.][0-9]+'
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[^\w\s\.]'         # 특수기호제거
    text = re.sub(pattern=pattern, repl='', string=text)
    text=re.sub('xa0','',text)
    text=text.replace('  ','')
    text=text.replace(' .','')
    text=text.replace('.....','')
    text=re.sub('①','',text)
    text=re.sub('②','',text)
    text=text.replace('..','')
    
    return text

In [6]:
df['text_clean']=df['text'].apply(lambda x: clean_str(x))

In [7]:
stopwords = """10,11,12,15,16,17,20,11월,1부,1분기,2010년,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2분기,3분기,4분기,4월,5월,6월,7월,가끔,가능,가능,가능성,가다,가동,가동,가드,가득,가량,가상현실,가운,가운데,가을,가장,가장자리,가정,가족,가치나,각각,각별,각종,감소,강민지,강수,강종민,강
하,강화,같다,같은,개교,개국,개나리,개념,개발,개방,개선,개양귀비,개월,개정,개정안,개최,거나,거래,거리,거부,거쳐,건립,건물,건설,건의,검사,검토,겨울,결과,결정,경남시장,경북시장,경우,경제,경찰,경칩,경향,경향신문,계곡,계속,계수,계정,계획,고려,고명진,고범준,고성,고속도로,고송민,고승민,곳곳,공간,공
감,공공,공공기관,공급,공기,공단,공동,공모,공모전,공무원,공사,공식,공연,공원,공항,공휴일,과정,과제,과천,과학기술,관계,관계자,관련,관리,관심,관측,관통,관한,광고,광장,광주시장,교수,교양,교육,교육과정,구간,구경,구독,구름,구름많고,구상,구석,구성,구역,구입,구조,구청,구축,국가,국내,국내외,국립,국립
환경과학,국립공원관리공단,국립생물자원관,국립생태원,국민,국방,국외,국장,국적,국제,국회,군락,군수,군청,권고,권역,권현구,규모,규칙,그간,그룹,그린,그림,극심,근로자,근본,근절,글로벌,금요일,금주,금
지,급여,기간,기고,기관,기념,기능,기대,기록,기반,기본,기사,기상,기상대,기상청,기술,기술대,기업,기여,기온,기운,기자,기자회견,기존,기준,기초,기해,기획,긴급,길이,김경,김광두,김성호,김용만,김인철,김재종,김진하,김태식,까지,꽃샘추위,나들,나들이,나무,나타,나타났다,나흘,나흘째,날림,날씨,날인,남남
서,남면,남부,남산,낭만,낮,내년,내달,내륙,내부,내외,내용,내일,네이버,노동,노력,노출,노후,논문,논의,놀이,농도,농림축산식품부,농장,누구,누드,누리집,뉴스,뉴시스,니스,다루다,다른,다만,다소,다시,다양,
다운,다음,다음주,단계,단기,단속,단지,단체,달라,닷새,당국,당부,당분간,당시,대공,대기,대박,대부분,대비,대상,대상,대응,대저,대전,대책,대체로,대통령,대표,대하,대학,대학교,대한,대한민국,대한항공,
대해,대해서,대행,대회,더위,데이트,도로,도록,도산,도시,도심,도움,도입,도출,도협회장,독도,독자,동구,동급,동남아,동물원,동민,동선,동아일보,동안,동영상,동진,되다,된다,둘째,둥지,드레스,드론,들이,등록,등장,디지털,따른,따오기,때문,떄문,또한,라며,라보,래시,로부터,류형,리기,리얼,리하,린다,마련,마리,마
을,마지막,만끽,만들기,만원,만큼,만화,말다,말했다,맑은,맑음,망신,맞이,맞춤,매년,매달,매우,매출,맨손,맺힌,머리,먹이,먼지,메인,면서,명품,모두,모든,모바일,모습,모양,모집,모텔,목록,목요일,목적,목표,
몸매,무단,무료,무상,무소속,무술년,문길주,문의,문제,문헌,문호,문화,물결,물고기,물관,물놀이,물든,물의,물이,물질,미녀,미디어,미만,미투데이,민간,민국,민원,밀양아리랑,바닥,바닷가,바람,바로,바로가기,박기량,박람회,박원순,박철규,반기,반기문,반대,발걸음,발견,발달,발령,발생,발언,발인,발전,발표,밤사
이,밤새,방문,방법,방식,방안,방침,방학,방향,배경,배병수,배포,배훈식,백로,법령,법화,벗꽃,벚꽃,변경,별로,병원,보가,보고,보고서,보급,보기,보다,보도,보이,보통,보호,복원,본격,본부,본회의,봄,봄날,봄비,봉사활동,부근,부담,부문,부제,부착,부터,부터 부터,북부,북상,북적,북주,분리,분석,분수,분야,브리핑,블랙리스트,블랙리스트,블록,비상,비키니,비판,비행,빗물,뽐내,뿌연,뿌옇,사거리,사고,사람,사랑,사례,사막,사무국,사무소,사실,사업,사업장,사용,사이,사이트,사장,사전,사진,사태,사항,사회,사흘,사흠,산간,산란,산림청,산업,산지,산책,산천어,삼척,상담,상림,상반기,상승,상춘객,상태,상한,상황,새끼,새로,새해,생산,생활,서도,서면,서명,서비스,서식,서울,서울대,서울시장,서지,서툴,선선,선언,선정,선택,설경,설계,설명,설비,설정,설치,성과,성매매,성분,성윤,성큼,세계,세대,세상,세영,세종로,센터,소개,소독,소만,소방,
소속,소식,소재,소통,소환,속담,손수제작물,수거,수도,수도권,수도권대기환경청,수립,수사,수산,수상,수요일,수용,수자원공사,수정,수준,수준진행,수출,수행,스님,스카이,스타,스타벅스,스탠드,스포츠,스폰서,
승인,시간,시간당,시내,시도,시민,시민행동,시민활동,시범,시설,시세,시스템,시의회,시작,시장,시정,시청,시티,시행,시험,식용,신고,신규,신속,신청,실무,실습,실시,실시간,실적,실정,실천,실험,실효,심각,심기,심사,썰매,쓰기,아가,아들,아래,아름,아름 다운,아시아,아지,아침, 아파트,악당,안강읍,안내,안병옥,
안정화,안지,알리,알피,앞바다,야구,양성,양식,어가,어서,어제,억원,언론,언제,업계,업데이트,업무,업무,업무추진비,업무협,업종,업체,었다,에서,여객선,여건,여기,여러분,여름,여름방학,여명,여부,여신,역대,역사,역할,연간,연구,연구원,연내,연속,연습,연일,연일읍,연출,연합,연합뉴스,연휴,영동,영상,영하,영향,예방,
예보,예산,예상,예정,오늘,오른쪽,오전,오후,온라인,올림픽,올해,외국인,요구,요청,요하,우려,우리,우리나라,우산,우선,우수,우장,운동,운동장,운영,원인,월드,월부,월요일,위반,위원,위원장,위원회,위촉장,위치,위해,위한,유강,유네스코,유명,유승민,유엔,유예,유의,유의 바란,유입,유투브,유튜브,유행,유형,육성,
윤성규,윤중로,으로,의견,의상,의원,의회,이곳,이기,이나,이낙연,이데일리,이동,이동면,이라,이루,이름,이물질,이미지,이번,이사장,이상,이상인,이야기,이에,이영환,이용,이유,이의,이전,이정선,이정섭,이젠,이진욱,이틀,
이하,이하늬,이행,이형기,이후,인간,인공,인근,인력양성,인사,인사말,인산인해,인양,인정,인증,인천시장,인턴,인파,일교,일대,일반,일보,일부,일요일,일이,일자리,임의,임태훈,임현정,입장,입장료,있다,자동,자락,자료,자웅,자체,자태,작년,작성,작업,작용,작품,잡고,잡기,장관,장관,장마전선,장병,장외,장점,장치,재
개,재차,재촉,재활용한,저곳,저녁,저작권,저장,적극,적발,적용,적응,전국,전날,전달,전략,전망,전망대,전면,전문,전문가,전반,전부도,전시,전시회,전체,전통,전해,전환,절기,절차,점검,점심,점차,정규직,정기,정도,정보,정보공개,정부,정비,정상,정상화,정원,정유년,정책,정책협의회,정확,제거,제공,제기,제도,제보,
제안,제외,제정,제출,제품,제한,조경규,조금,조기,조명,조명래,조사,조성,조정,조치,조현아,족구,종록,종원,종합,주가,주간,주년,주말,주무관,주민,주변,주상,주시 반영,주연,주요,주위,주의,주장,주제,주차장,주택,준비,중구,중단,중복,중복,중부,중부지방,중심,중앙,중이,중인,중점,증가,지고,지구,지금,지나,지난,지
난달,지난해,지방,지방선거,지속,지식,지역,지역별,지원,지자체,지적,지정,지청,지침,지하철,직원,직접,진짜,진행,진화,질문,질의응답,질환,집중,쯔위,차관,차례,차용,차차,참가,참가자,참고,참석,참여,참여하,
창업,채널,채용,채취,채팅,책임,처리,처분,처음,천시,천은사,철쭉,첫날,청계천,청명,청사,청소,청소년,청와대,청장,청주,체결,체계,체납,체육,체제,체험,초과,촉구,총리,총선,총회,촬영,최강,최고,최근,최다,최대,최대한,최문순,최원혁,최저,최진,최창호,최초,추가,추상,추석,추억,추연,추진,추진 ,축구,축제,춘분,출근
길,출범,취급,취소,측정,치가,치어리더,침대,카메라,캠퍼스,캠페인,컨설팅,코리아,크게,크기,클릭,타워,탐방,태안,태양,태현,터트려,터트린,테마,토론,토론회,토요일,통과,통신사,통합,통해,통행,투자,투표,투표,튤립,트렌드,트위터,특별,특보,특위,특조위,특징,파악,파크,판단,패럴림픽,퍼포먼스,페이스북,평가,평균,평년,평년기온,평택,포럼,포토,포함,폭스바겐,표시,표준,표지,푸른,풍경,프레,프로그램,플러스,피어,피하,피해,피해자,필요,하게,하경민,하고,하기,하나,하늘,하도,하라,하루,하루종일,하루종일,하반기,하였다,학교,학생,학원,한겨레,한경닷컴,한국,한다,한목소리,한반도,한윤,한펀,할인,합동,합의,항일,해결,해경,해당,해법,해상,해양,해외,해운대해수욕장,해제,해체,해피,햇살,했다,행동,행사,행위,행정,행정 공공기
관,행정부,향후,허가,허브,허용,헤럴드,헤럴드경제,혁신,현대,현상,현실,현장,현재,현지,현충일,현황,협력,협력방안,협악,협회,형형색색,호기,호인,호텔,홈페이지,홍보,화려,화보,화요일,화제,화창,화학물질안전원,확대,확률,확보,확산,확인,환경,환경과학,환경기술,환경단체,환경미화원,환경백서,환경부,환경분야,환경운동,환경재단,환경정책,환경청,환자,활동,활성화,활약,활용,활짝,황금,회관,회원,회의,효과,후보,후회,훈련,휴가,휴업,휴일,흥부가,는,를,을,및,것,뿐,따라, 또는, 라고, 라는"""

In [8]:
#문장단위로 나누기
def tidy_sentences(section):
    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])(\s*\n|\.|;)|(?<=다)\.)\s*')
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'
    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return sentences

In [9]:
df['sentence']=df['text_clean'].apply(lambda x: tidy_sentences(x))

In [10]:
df.head(2)

,date,department,title,text,text_clean,sentence
0,2020-09-29,생물다양성과,공동참고철새도래지 10개소 89종 28277개체 확인,['농식품부는 최근 겨울철새 도래현황 조사 결과환경부 대표적인 겨울철새인 오리기러기...,농식품부는 최근 겨울철새 도래현황 조사 결과환경부 대표적인 겨울철새인 오리기러기 도...,[농식품부는 최근 겨울철새 도래현황 조사 결과환경부 대표적인 겨울철새인 오리기러기 ...
1,2020-09-29,환경보건정책과,가습기살균제 피해 300명 추가 인정총 3284명 지원,['제19차 가습기살균제 피해구제위원회 개최 특별법 개정 후 첫 회의 개정법에 따른...,제차 가습기살균제 피해구제위원회 개최 특별법 개정 후 첫 회의 개정법에 따른 신속심...,[제차 가습기살균제 피해구제위원회 개최 특별법 개정 후 첫 회의 개정법에 따른 신속...


In [11]:
df['text_clean_stop']=df['text_clean'].apply(lambda x: [i for i in mecab.morphs(x) if i not in stopwords])

In [455]:
stop_text=[]
for  s  in  df['sentence']:
    for i in s:
        for j in mecab.morphs(i):
            if j not in stopwords:
                stop_text.append(j)

In [458]:
counter_stop=Counter(stop_text)
counter_stop.most_common()[:10]

[('.', 34775),
 ('배출', 3577),
 ('미세먼지', 3090),
 ('밝혔', 2417),
 ('폐기물', 2232),
 ('안전', 1974),
 ('저감', 1961),
 ('될', 1905),
 ('오염', 1898),
 ('함께', 1707)]

In [330]:
mecab=Mecab()

In [331]:
#pos tagging
df['pos']=df['sentence'].apply(lambda x: [mecab.pos(i) for i in x])

In [335]:
def generate_ngrams(pos_list, n):
    ngrams = list(zip(*[pos_list[i:] for i in range(n)]))
    return ngrams

In [336]:
df['pos_bigram']=df['pos'].apply(lambda x:[generate_ngrams(i,2) for i in x])

In [377]:
unigram_list=[]
for i in df['pos']:
    for j in range(len(i)):
        for k in range(len(i[j])):
            unigram_list.append(i[j][k])

In [378]:
counter=Counter(unigram_list)

In [471]:
counter.most_common()[:10]

[(('을', 'JKO'), 38411),
 (('.', 'SF'), 34250),
 (('하', 'XSV'), 29374),
 (('의', 'JKG'), 26752),
 (('에', 'JKB'), 26749),
 (('다', 'EF'), 23338),
 (('를', 'JKO'), 22418),
 (('등', 'NNB'), 17407),
 (('으로', 'JKB'), 16875),
 (('는', 'ETM'), 15977)]

1. 생물
2. 배출
3. 미세먼지
4. 생태
5. 공원

In [380]:
bigram_list=[]
for i in df['pos_bigram']:
    for j in range(len(i)):
        for k in range(len(i[j])):
            bigram_list.append(i[j][k])

In [382]:
counter2=Counter(bigram_list)

In [383]:
counter2.most_common()

[((('다', 'EF'), ('.', 'SF')), 21864),
 ((('하', 'XSV'), ('고', 'EC')), 7258),
 ((('하', 'XSV'), ('여', 'EC')), 6790),
 ((('하', 'XSV'), ('는', 'ETM')), 6718),
 ((('이', 'VCP'), ('다', 'EF')), 5942),
 ((('수', 'NNB'), ('있', 'VV')), 4773),
 ((('한다', 'XSV+EF'), ('.', 'SF')), 4604),
 ((('했', 'XSV+EP'), ('다', 'EF')), 4594),
 ((('적', 'XSN'), ('으로', 'JKB')), 3648),
 ((('월', 'NNG'), ('일', 'NNG')), 3534),
 ((('고', 'EC'), ('있', 'VX')), 3209),
 ((('에', 'JKB'), ('대한', 'VV+ETM')), 2840),
 ((('하', 'XSV'), ('기', 'ETN')), 2695),
 ((('할', 'XSV+ETM'), ('수', 'NNB')), 2683),
 ((('등', 'NNB'), ('을', 'JKO')), 2682),
 ((('계획', 'NNG'), ('이', 'VCP')), 2409),
 ((('밝혔', 'VV+EP'), ('다', 'EF')), 2408),
 ((('적', 'XSN'), ('인', 'VCP+ETM')), 2384),
 ((('예정', 'NNG'), ('이', 'VCP')), 2357),
 ((('국립', 'NNG'), ('공원', 'NNG')), 2322),
 ((('월', 'NNG'), ('일', 'NR')), 2263),
 ((('에', 'JKB'), ('따라', 'VV+EC')), 2095),
 ((('것', 'NNB'), ('으로', 'JKB')), 2028),
 ((('할', 'XSV+ETM'), ('계획', 'NNG')), 2016),
 ((('환경부', 'NNG'), ('장관', 'NNG')), 1946

1. 국립_공원
2. 생물_자원
3. 물_관리
4. 화학_물질
5. 오염_물질
6. 생태_계
7. 다양_성
8. 대기_오염
9. 지속_가능
10. 생물_다양

In [399]:
df['morph']=df['sentence'].apply(lambda x: [mecab.morphs(i) for i in x])

In [401]:
unigram_list_m=[]
for i in df['morph']:
    for j in range(len(i)):
        for k in range(len(i[j])):
            unigram_list_m.append(i[j][k])

In [402]:
counter_u_m=Counter(unigram_list_m)

In [457]:
counter_u_m.most_common()[:10]

[('을', 39414),
 ('.', 36217),
 ('하', 34110),
 ('는', 31746),
 ('이', 27419),
 ('에', 26899),
 ('의', 26836),
 ('다', 24387),
 ('를', 22566),
 ('등', 17495)]

In [13]:
# TF-IDF matrix를 생성한다.
tf_vector = TfidfVectorizer(max_features = 100)
tfidf = tf_vector.fit_transform(stop_text)
print(tfidf.shape)

vocab = tf_vector.get_feature_names()
print(vocab[:])

NameError: name 'TfidfVectorizer' is not defined

# LDA

In [470]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

lda = LDA(n_components=20,
          max_iter=20,
          learning_method='online',
          random_state=100)
lda_output = lda.fit_transform(tfidf)

In [476]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, tfidf, tf_vector, mds='tsne')
pyLDAvis.display(vis)

In [474]:
pip install pyLDAvis

     |████████████████████████████████| 1.6 MB 2.5 MB/s eta 0:00:01
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=ceaed0e525c2a1b94eaa9f51415cee06c97ce892be57e5eb2442d186f11dc3fd
  Stored in directory: /Users/bomi/Library/Caches/pip/wheels/3b/fb/41/e32e5312da9f440d34c4eff0d2207b46dc9332a7b931ef1e89
Successfully built pyLDAvis
Note: you may need to restart the kernel to use updated packages.
